In [1]:
"""Demo notebook."""

import os
from niceview.utils.raster import geo_ref_raster
from niceview.utils.mask import sparse_npz_to_array
from niceview.utils.tools import txt_to_list, select_col_from_name, normalize_nonzero_to_range
from niceview.utils.cell import get_nuclei_pixels, paint_regions
import numpy as np
import pandas as pd
import scipy
import cv2

In [2]:
# configurations
DATA_PATH = '../examples/data/'

In [3]:
# args
file = {
    'cells-gene-names': 'gt-iz-p9-rep2-cells-gene-names.txt',
    'cells-gene': 'gt-iz-p9-rep2-cells-gene.npz',
    'cells-info': 'gt-iz-p9-rep2-cells-info.csv',
    'mask': 'gt-iz-p9-rep2-mask.npz',
}

mask_filtered_relabeled_name = 'gt-iz-p9-rep2-mask-filtered-relabeled.npz'  # variable
img_name = 'gt-iz-p9-rep2-img.tiff'  # variable
img_gis_name = 'gt-iz-p9-rep2-img-gis.tiff'  # variable
img_masked_name = 'gt-iz-p9-rep2-img-masked.png'  # variable
img_masked_gis_name = 'gt-iz-p9-rep2-img-masked-gis.tiff'  # variable
cell_info_name = 'gt-iz-p9-rep2-cells-info.csv'  # variable

selected_gene_id = 'ENSG00000065534'

In [4]:
# read data
mask = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-mask.npz'))
cell_pos = pd.read_csv(DATA_PATH + 'gt-iz-p9-rep2-cells-info.csv')[['x', 'y']].values
cells_gene = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene.npz'))
gene_names = txt_to_list(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene-names.txt'))

# get nuclei pixels
gene = select_col_from_name(cells_gene, gene_names, selected_gene_id)
matched_regions = get_nuclei_pixels(mask, cell_pos)

# relabel mask
mask_filtered_relabeled = paint_regions(mask.shape, matched_regions, cell_colors_list=gene + 1e-6)
mask_filtered_relabeled = scipy.sparse.csr_matrix(mask_filtered_relabeled.data)

# save relabeled mask
scipy.sparse.save_npz(DATA_PATH + 'gt-iz-p9-rep2-mask-filtered-relabeled.npz', mask_filtered_relabeled)

# read relabeled mask
mask_filtered_relabeled = sparse_npz_to_array(os.path.join(DATA_PATH, mask_filtered_relabeled_name))

# normalize relabeled mask
mask_filtered_relabeled_norm = normalize_nonzero_to_range(mask_filtered_relabeled)

In [5]:
# image gis
img_gis_path = geo_ref_raster(
    os.path.join(DATA_PATH, img_name),
    os.path.join(DATA_PATH, img_gis_name),
    overwrite=False,
)

File ../examples/data/gt-iz-p9-rep2-img-gis.tiff already exists.


In [63]:
def mask_overlay_image(img_path, mask, dst_path, mask_opacity=0.3, colormap='jet', overwrite=False):
    """Overlay mask on image and save to file.

    Args:
        img_path (str): path to image.
        mask (np.ndarray): mask array of shape (row, col), background value is 0.
        dst_path (str): destination path.
        mask_opacity (float): opacity of mask.
        colormap (str): colormap of mask.
        overwrite (bool): whether to overwrite existing file.

    Returns:
        str: destination path.
    """
    if os.path.exists(dst_path) and not overwrite:
        print(f'File {dst_path} already exists.')
        return dst_path
    if colormap == 'jet':
        cmap = cv2.COLORMAP_JET
    
    # mask image
    mask_img = cv2.cvtColor(mask.astype(np.uint8), cv2.COLOR_BGR2RGB)
    mask_img = cv2.applyColorMap(mask_img, cmap)
    mask_blend = cv2.bitwise_and(mask_img, mask_img, mask=mask.astype(np.uint8))
    
    # background image
    bkgd_img = cv2.imread(img_path)
    bkgd_blend = cv2.bitwise_and(bkgd_img, bkgd_img, mask=mask.astype(np.uint8))
    inv_mask = (mask == 0).astype(np.uint8)
    bkgd = cv2.bitwise_and(bkgd_img, bkgd_img, mask=inv_mask)
    
    # overlay
    mask_ovelay = cv2.addWeighted(mask_blend, mask_opacity, bkgd_blend, 1.0 - mask_opacity, 0)
    whole_img = cv2.addWeighted(mask_ovelay, 1.0, bkgd, 1.0, 0)
    
    # save
    cv2.imwrite(dst_path, whole_img)
    return dst_path

In [64]:
# image with mask overlay witout geo-referencing
img_masked_path = mask_overlay_image(
    os.path.join(DATA_PATH, img_name),
    mask_filtered_relabeled_norm,
    os.path.join(DATA_PATH, img_masked_name),
    mask_opacity=0.5,
    overwrite=True,
)

In [62]:
# image with mask overlay with geo-referencing
img_masked_gis_path = geo_ref_raster(
    img_masked_path,
    os.path.join(DATA_PATH, img_masked_gis_name),
    overwrite=True,
)